# Libs

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import ipywidgets as widgets
from ipywidgets import fixed

In [2]:
%config IPCompleter.greedy=True
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Lista dos Fundos Imobiliarios

In [3]:
# o header abaixo serve para mascarar a requisição semelhante a um web browser
header = {"User-Agent": "Chrome/84.0.4147.89","X-Requested-With": "XMLHttpRequest"}

# URL com todos os FIIs listados
url = 'https://fundamentus.com.br/fii_resultado.php'

# Requests via HTTP
r = requests.get(url, headers = header)

# Leitura do HTML entregue
fii = pd.read_html(r.text,decimal=',',thousands='.')[0]


In [4]:
# o índice [0] contem toda a tabela dos fundos
fundos = fii.copy()

# Dividend Yield vem como string. É necessário remover '%' e trocar ',' por '.'
fundos['Dividend yield 2']= fundos['Dividend Yield'].str.replace('%','').str.replace(',','.').astype(float)
fundos['Dividendo/cota'] = fundos['Cotação']*fundos['Dividend yield 2']/100

fundos.loc[fundos['Segmento'].isna(), 'Segmento'] = 'Outros'

fundos

ValueError: could not convert string to float: '1.397.74'

# Obtendo o IPCA

In [9]:
# OBTER IPCA acumulado 12 MESES

r = requests.get('https://www.melhorcambio.com/ipca',headers = header,verify=False)

# web scrapying para "quebrar o conteúdo do html"
soup = bs(r.content,'html.parser')

ipca = soup.find_all('input',{'id':'inp_calc_3'})

ipca = float(ipca[0]['value'].replace(',','.'))
ipca

10.38

# Obtendo o SPREAD do IPCA

- o "*spread*" é a diferença que o título mais longo do Tesouro está pagando de diferencial em relação ao IPCA
- Exemplo: IPCA+2050 = IPCA + 5% -> os 5% é o spread do titulo

In [10]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
# função que faz o crawling dos dados do site do Banco Central

def busca_titulos_tesouro_direto():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
    df['Data Base']       = pd.to_datetime(df['Data Base'], dayfirst=True)
    return df

In [12]:
# Transformar o título em dataframe

titulos = busca_titulos_tesouro_direto()
titulos.sort_index(inplace=True)

In [13]:
# Obter o IPCA+ longo

vencimento = titulos.loc[(titulos['Tipo Titulo'] == 'Tesouro IPCA+'),'Data Vencimento'].max()
ipca_longo = titulos.loc[(titulos['Tipo Titulo'] == 'Tesouro IPCA+') & (titulos['Data Vencimento'] == vencimento)]

In [14]:
# Obter o spread que o título mais longo está pagando 

ipca_longo.sort_values('Data Base',ascending = False, inplace = True)
spread_ntnb = ipca_longo.iloc[0,3]
spread_ntnb

5.52

# Modelo de Gordon

Modelo de Gordon = $( Dividendo ) / Wacc $

$ Wacc = Spread.NTNB + IPCA + Spread.Mercado$

- **Tijolo** -> `IPCA = 0` pois a inflação já é reposta pelo patrimônio e reajuste dos contratos
- **CRI** -> `IPCA <> 0` (retorno é nominal)

- `Spread NTN-B`: spread do título IPCA mais longo
- `IPCA`: indice acumulado dos ultimos 12 meses
- `Spread Mercado`: quanto o mercado está precificando acima do tesouro mais longo (geralmente fica em torno de 2% acima)
    
- Iremos usar o `WIDGETS` do Python para simular o `SPREAD_MERCADO`

In [15]:
# Criar 2 colunas com os dados já obtidos

fundos['NTN-B'] = spread_ntnb
fundos['IPCA'] = ipca

NameError: name 'fundos' is not defined

In [33]:
fundos.loc[(fundos['Papel'].str.contains('OF')) | (fundos['Papel'].str.contains('FF'))]

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média,Dividend yield 2,Dividendo/cota,NTN-B,IPCA
4,AFOF11,Títulos e Val. Mob.,182.16,"2,20%","0,00%",1.69,66875300,6731,0,0.00,0.00,"0,00%","0,00%",0.00,0.000000,4.08,8.06
19,BCFF11,Títulos e Val. Mob.,85.38,"2,93%","4,16%",1.01,2153160000,4354060,0,0.00,0.00,"0,00%","0,00%",4.16,3.551808,4.08,8.06
29,BPFF11,Outros,79.98,"5,03%","5,04%",0.95,359296000,1055790,0,0.00,0.00,"0,00%","0,00%",5.04,4.030992,4.08,8.06
45,CPFF11,Títulos e Val. Mob.,83.14,"2,53%","4,02%",1.00,330379000,1336260,0,0.00,0.00,"0,00%","0,00%",4.02,3.342228,4.08,8.06
47,CRFF11,Títulos e Val. Mob.,73.54,"4,19%","4,50%",0.80,50745500,72556,0,0.00,0.00,"0,00%","0,00%",4.50,3.309300,4.08,8.06
70,FFCI11,Lajes Corporativas,182.03,"3,65%","0,00%",0.93,671817000,0,10,14522.70,700.80,"4,83%","7,34%",0.00,0.000000,4.08,8.06
79,FMOF11,Lajes Corporativas,70.01,"1,45%","1,53%",0.53,35565600,2815,2,2106.94,174.95,"8,30%","39,48%",1.53,1.071153,4.08,8.06
87,GCFF11,Títulos e Val. Mob.,97.20,"1,13%","4,64%",1.01,32911700,79976,0,0.00,0.00,"0,00%","0,00%",4.64,4.510080,4.08,8.06
100,HFOF11,Títulos e Val. Mob.,97.91,"6,32%","4,93%",1.02,2109670000,3428760,0,0.00,0.00,"0,00%","0,00%",4.93,4.826963,4.08,8.06
103,HGFF11,Títulos e Val. Mob.,83.48,"4,68%","5,51%",0.86,239053000,688840,0,0.00,0.00,"0,00%","0,00%",5.51,4.599748,4.08,8.06


In [40]:
def modelo_gordon(fiis,spread_mercado, segmento):
    # filtro para zerar o IPCA de todos os FIIs que não sao de CRI
    fiis.loc[~(fiis['Segmento']=='Títulos e Val. Mob.')|
            (fiis['Papel'].str.contains('OF')) |
            (fiis['Papel'].str.contains('FF')) ,'IPCA'] = 0
    fiis['SPREAD'] = spread_mercado
    fiis['WACC'] = (fiis['NTN-B'] + fiis['IPCA'] + fiis['SPREAD'])/100
    fiis['ValorJusto'] = round(fiis['Dividendo/cota'] / fiis['WACC'],2)
    fiis['Sinal'] = 'Acima valor justo'
    fiis.loc[(fiis['ValorJusto']-fiis['Cotação']) >= 0 ,'Sinal'] = 'Abaixo valor justo'
    a = fiis[['Papel','Segmento','Cotação','ValorJusto','IPCA','Sinal']]
    a = a.loc[(a['Sinal'] == 'Abaixo valor justo') & (a['Segmento'] == segmento)]
    print(a)

In [41]:
# Slider do spread mercado
valor_segmento = widgets.Dropdown(options = fundos['Segmento'].unique(),
        value = fundos['Segmento'].iat[0],description = 'Segmento:', disable = False)

valor_spread = widgets.FloatSlider(value=2,min=0,max=2.5,step=0.1,description='SpreadMercado:',
    disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='.1f')
        
# WIDGET CREATED      
widgets.interactive(modelo_gordon,fiis = fixed(fundos),spread_mercado = valor_spread,segmento = valor_segmento)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='SpreadMercado:', max=2.5, r…